In [1]:
import sys
import time
import datetime
import os
import tensorflow as tf
import argparse
import json
import socket
from PIL import Image
from util import *
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
logPath = "training/"
snapshotPath = "snapshots/"

In [3]:
with open("hyperParams.json") as f:
    instanceParams = json.load(f)

instanceParams

{u'baseLR': 1e-05,
 u'batchSize': 2,
 u'borderThicknessH': 0.01,
 u'borderThicknessW': 0.01,
 u'boundaryAlpha': 6.5,
 u'dataset': u'kitti2012',
 u'flowScale': 20.0,
 u'gradParams': {u'robustness': 0.46, u'scale': 255.0, u'weight': 6.408},
 u'instanceName': u'unsupFlownet',
 u'iterations': 500000,
 u'lossComponents': {u'asymmetricSmooth': True,
  u'backward': False,
  u'boundaries': False,
  u'gradient': False,
  u'smooth2nd': False},
 u'photoParams': {u'robustness': 0.53, u'scale': 360.0},
 u'printFreq': 100,
 u'resnet': False,
 u'smooth2ndParams': {u'robustness': 0.21, u'scale': 1.0, u'weight': 0.53},
 u'smoothOccParams': {u'robustness': 0.9, u'scale': 1.8},
 u'smoothParams': {u'robustness': 0.28, u'scale': 3.5, u'weight': 0.64},
 u'snapFreq': 5000,
 u'snapshotFreq': 10000,
 u'weightDecay': 0.0}

In [4]:
printFrequency = instanceParams["printFreq"]
snapshotFrequency = instanceParams["snapFreq"]
batchSize = instanceParams["batchSize"]

iterations = instanceParams["iterations"]
baseLearningRate = instanceParams["baseLR"]
learningRate = baseLearningRate
snapshotFrequency = instanceParams["snapshotFreq"]



from dotmap import DotMap
arg = DotMap()
arg.logDev = False
arg.resume = 'y'
resume, startIteration, snapshotFiles = checkResume(snapshotPath,logPath, arg)
iterations = 1000 * 10

No snapshots found, training from scratch


Below is the code used to generate train image txt files from DAVIS's train.txt.

In [5]:
with tf.device("/gpu:0"):
    trainingData = TrainingData(batchSize,instanceParams)

with tf.device("/gpu:0"):
    # init
    with tf.variable_scope("netShare"):
        networkBodyF = NetworkBody(trainingData,instanceParams)
    with tf.variable_scope("netShare",reuse=True):
        networkBodyB = NetworkBody(trainingData,instanceParams,flipInput=True)

    trainingLoss = TrainingLoss(instanceParams,networkBodyF,networkBodyB,trainingData)
    solver,learningRateTensor = attachSolver(trainingLoss.loss)

    # loss scheduling
    recLossBWeightTensor = trainingLoss.recLossBWeight

Reading Tensor("ImagePairData/image_data_reader/Placeholder:0", dtype=string, device=/device:GPU:0)
Reading Tensor("ImagePairData/image_data_reader_1/Placeholder:0", dtype=string, device=/device:GPU:0)
Reading Tensor("ImagePairData/image_data_reader_2/Placeholder:0", dtype=string, device=/device:GPU:0)


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [6]:
# merge summaries
merged = tf.summary.merge_all()
# saver
saver = tf.train.Saver(max_to_keep=0)

### Training

In [ ]:
# start
with sessionSetup(arg) as sess:
#     if resume:
#         saver.restore(sess,snapshotPath+snapshotFiles[-1][:-6])
#     else:
#         sess.run(tf.initialize_all_variables())
    saver.restore(sess,
                  '../model_download_scripts/photometric_smoothness/weights/iter_0000000000500000.ckpt')

    trainingData.dataQueuer.start_queueing(sess)

    #start summary writer
    summary_writer = tf.summary.FileWriter(logPath, sess.graph)

    #run
    lastPrint = time.time()
    for i in range(500000, 500000 + iterations + 10):
        # scheduled values
        learningRate = learningRateSchedule(baseLearningRate, i)
        recLossBWeight = unsupLossBSchedule(i)

         #run training
        feed_dict = {
            learningRateTensor: learningRate,
            recLossBWeightTensor: recLossBWeight,
        }
        summary,result,totalLoss = sess.run([merged,solver,trainingLoss.loss], feed_dict=feed_dict)

        if (i+1) % printFrequency == 0:
            timeDiff = time.time() - lastPrint
            itPerSec = printFrequency/timeDiff
            remainingIt = iterations - i
            eta = remainingIt/itPerSec
            print("Iteration "+str(i+1)+": loss: "+str(totalLoss)+", iterations per second: "+str(itPerSec)+", ETA: "+str(datetime.timedelta(seconds=eta)))+", lr: "+str(learningRate)

            summary_writer.add_summary(summary,i+1)
            summary_writer.flush()
            lastPrint = time.time()

        if (i+1) % snapshotFrequency == 0:
            saver.save(sess,"snapshots/iter_"+str(i+1).zfill(16)+".ckpt")

        sys.stdout.flush()

    #close queing
    trainingData.dataQueuer.close(sess)

INFO:tensorflow:Restoring parameters from ../model_download_scripts/photometric_smoothness/weights/iter_0000000000500000.ckpt
Iteration 500100: loss: [[49.266235]
 [30.332296]], iterations per second: 1.16948768904, ETA: -5 days, 3:35:28.465863, lr: 1.25e-06
Iteration 500200: loss: [[47.7024  ]
 [38.763184]], iterations per second: 1.43089391132, ETA: -4 days, 0:50:17.649120, lr: 1.25e-06
Iteration 500300: loss: [[34.976307]
 [63.656944]], iterations per second: 1.43221282845, ETA: -4 days, 0:54:23.309348, lr: 1.25e-06
Iteration 500400: loss: [[77.419106]
 [84.77728 ]], iterations per second: 1.44917843395, ETA: -4 days, 2:00:02.057751, lr: 1.25e-06
Iteration 500500: loss: [[28.02604 ]
 [10.730354]], iterations per second: 1.43647055846, ETA: -4 days, 1:08:58.772610, lr: 1.25e-06
Iteration 500600: loss: [[52.564774]
 [30.86668 ]], iterations per second: 1.438229011, ETA: -4 days, 1:14:46.730661, lr: 1.25e-06
Iteration 500700: loss: [[34.29627 ]
 [23.786419]], iterations per second: 1.4

### Testing

In [ ]:
datasetRoot = '../example_data/'
frame0Path = '../example_data/datalists/train_im0.txt'
frame1Path = '../example_data/datalists/train_im1.txt'
desiredHeight = 480
desiredWidth = 854

In [ ]:
with open(frame0Path) as f:
    imagePairs0 = [datasetRoot+x[:-1] for x in f.readlines()]
with open(frame1Path) as f:
    imagePairs1 = [datasetRoot+x[:-1] for x in f.readlines()]

In [ ]:
testData = TestData(imagePairs0,imagePairs1,1,desiredHeight,desiredWidth)
#393

In [ ]:
with tf.device("/gpu:0"):
    with tf.variable_scope("netShare"):
        networkBody = NetworkBody(testData,instanceParams)
flowFinal = networkBody.flows[0]

In [ ]:
flowViz = flowToRgb(flowFinal)
transformGrid = flowTransformGrid(flowFinal)
mean = [0.407871, 0.457525, 0.481094]
mean = tf.expand_dims(tf.expand_dims(tf.expand_dims(mean,0),0),0)
# warped = flowWarp(testData.frame1["rgb"]+mean,flowFinal)

In [ ]:
saver = tf.train.Saver()

# config tensorflow
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
iterations = 510000

In [ ]:
test_length = 70
result_dir = 'results'
with tf.Session(config=config) as sess:
    saver.restore(sess,"snapshots/iter_"+str(iterations).zfill(16)+".ckpt")

    # run
    lastPrint = time.time()
    for i in range(697, 697 + test_length):
        feed_dict = {
            testData.im0File: imagePairs0[i],
            testData.im1File: imagePairs1[i],
        }
        result = sess.run([flowViz,testData.height,testData.width],feed_dict=feed_dict)
        h = result[1]
        w = result[2]

        arr = np.maximum(np.minimum(np.asarray(result[0]),1), 0)
        arr = np.squeeze(np.asarray(arr*255,np.uint8))
        im = Image.fromarray(arr[:h,:w,:])
        im.save("{}/{}.png".format(result_dir, str(i).zfill(3)))